In [ ]:
import torch
from collections import Counter
from torch.utils.data import DataLoader
from torchsummary import summary
import numpy as np
from torch import nn, optim

import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler


In [ ]:

class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()
        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}
        self.words_indexes = [self.word_to_index[w] for w in self.words]
        self.sequence_length=4
    def load_words(self):
        self.text= open("lyrics2.txt","r")
        self.text=str(self.text.read())
        return self.text.split(' ')
    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)
    def __len__(self):
        return len(self.words_indexes) - self.sequence_length
    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1]),
        )

In [ ]:
dataset=Dataset()
batch_size = 128
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [ ]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 400
        self.embedding_dim = 400
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
            
        )
        
        self.fc = nn.Linear(self.lstm_size, n_vocab)


    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        
        logits = (self.fc(output))

        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).cuda(),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).cuda())

In [ ]:
model=Model(dataset)
print(model.cuda())

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
val_loss_min=np.Inf

for epoch in range(10):
        state_h, state_c = model.init_state(4)
        state_h, state_c= state_h.cuda(), state_c.cuda()
        model.train()
        for batch, (x, y) in enumerate(train_loader):
            x,y=x.cuda(),y.cuda()
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            train_loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()
            

            train_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(),6)
            optimizer.step()
            if (batch%1000==0):
                print({ 'epoch': epoch, 'batch': batch,  'training loss': train_loss.item() })
                torch.save(model.state_dict(), "lyricstrain.pt")
    
        
        model.eval()
        for batch, (x, y) in enumerate(validation_loader):
                x,y=x.cuda(),y.cuda()
                optimizer.zero_grad()

                y_pred, (state_h, state_c) = model(x, (state_h, state_c))
                val_loss = criterion(y_pred.transpose(1, 2), y)

                state_h = state_h.detach()
                state_c = state_c.detach()
                optimizer.step()
                if (batch%1000==0):
                    print({ 'epoch': epoch, 'batch': batch, ' validation loss': val_loss.item(), 'training loss': train_loss.item() })
                if(val_loss.item()<val_loss_min):
                    
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,val_loss.item()))
                    val_loss_min=val_loss.item()
                    torch.save(model.state_dict(), "lyricsval.pt")


In [ ]:
model.load_state_dict(torch.load("lyricsval.pt"))

In [ ]:
def predict(dataset, model, text, next_words=50):
    words = text.split(' ')
    model.eval()

    state_h, state_c = model.init_state(len(words))
    state_h, state_c= state_h.cuda(), state_c.cuda()


    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]]).cuda()
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))
        word_weights = torch.nn.functional.softmax(y_pred.squeeze().data.div(0.64).exp().cpu(),dim=0).cpu().detach()
        word_idx = torch.multinomial(word_weights, 1)[0]
        #print(word_idx)
        words.append(dataset.index_to_word[word_idx.item()])
    return (words)
    


In [ ]:
word_list = (predict(dataset, model, text='zindagi'))
strword=str()
for i in word_list:
    strword+=" "+i
print(strword)